In [203]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [204]:
import sys
sys.path.insert(0, '..//')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from e2slib.utillib import dummy_data, functions
from src.data import manipulations, data_preperation
from e2slib.structures import enums, datetime_schema
from datetime import datetime

In [205]:
demand = dummy_data.create_dummy_site_demand(2022)
carbon = dummy_data.generate_dummy_carbon_profile(2022)
price = dummy_data.generate_dummy_price_profile(2022)

In [206]:
functions.add_time_features(demand)

,Site energy [kWh],Hour,Day of week,Day of year,Month,Year,Weekday flag,Half-hour,Date,Week,season,season_number
2022-01-01 00:00:00+00:00,10,0,5,1,1,2022,weekend,0,2022-01-01,52,WINTER,1
2022-01-01 00:30:00+00:00,9,0,5,1,1,2022,weekend,1,2022-01-01,52,WINTER,1
2022-01-01 01:00:00+00:00,2,1,5,1,1,2022,weekend,2,2022-01-01,52,WINTER,1
2022-01-01 01:30:00+00:00,0,1,5,1,1,2022,weekend,3,2022-01-01,52,WINTER,1
2022-01-01 02:00:00+00:00,10,2,5,1,1,2022,weekend,4,2022-01-01,52,WINTER,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 21:30:00+00:00,7,21,5,365,12,2022,weekend,43,2022-12-31,52,WINTER,1
2022-12-31 22:00:00+00:00,2,22,5,365,12,2022,weekend,44,2022-12-31,52,WINTER,1
2022-12-31 22:30:00+00:00,3,22,5,365,12,2022,weekend,45,2022-12-31,52,WINTER,1
2022-12-31 23:00:00+00:00,4,23,5,365,12,2022,weekend,46,2022-12-31,52,WINTER,1


In [207]:
cols = [
        datetime_schema.DateTimeSchema.DAYOFWEEK,
        datetime_schema.DateTimeSchema.HALFHOUR
    ]

In [208]:
demand.columns[0]

'Site energy [kWh]'

In [209]:
df = manipulations.create_seasonal_average_week(enums.Season.SUMMER, demand)
df

,Site energy [kWh]
0.000000,4.846154
0.020833,5.076923
0.041667,4.769231
0.062500,4.846154
0.083333,3.846154
...,...
6.895833,6.000000
6.916667,4.538462
6.937500,5.461538
6.958333,4.692308


In [210]:
import plotly.express as px

# df = px.data.gapminder().query("country=='Canada'")


fig = px.line(df, x=df.index, y="Site energy [kWh]", title='Life expectancy in Canada')
fig.show()


In [211]:
data = {'A': [10, 15, 12, 14, 20, 8, 11, 13, 16, 5, 1000],
        'B': [50, 55, 52, 54, 60, 48, 51, 53, 56, 45, 2000],
        'C': [100, 150, 120, 140, 200, 80, 110, 130, 160, 50, 3000]}

df = pd.DataFrame(data)

# Add outliers
df.loc[3, 'A'] = 500
df.loc[6, 'B'] = 3000
df.loc[9, 'C'] = 5000

In [212]:
print(df)
outlier_remover = data_preperation.OutlierRemover()
outliers_removed_df = outlier_remover.data_cleaner(data=df)

print("\nDataFrame with outliers removed:")
print(outliers_removed_df)

       A     B     C
0     10    50   100
1     15    55   150
2     12    52   120
3    500    54   140
4     20    60   200
5      8    48    80
6     11  3000   110
7     13    53   130
8     16    56   160
9      5    45  5000
10  1000  2000  3000

DataFrame with outliers removed:
       A     B      C
0   10.0  50.0  100.0
1   15.0  55.0  150.0
2   12.0  52.0  120.0
3    NaN  54.0  140.0
4   20.0  60.0  200.0
5    8.0  48.0   80.0
6   11.0   NaN  110.0
7   13.0  53.0  130.0
8   16.0  56.0  160.0
9    5.0  45.0    NaN
10   NaN   NaN    NaN


In [214]:
fillMissing = data_preperation.FillMissingData()
filled_na_df = fillMissing.data_cleaner(data=outliers_removed_df,func='fillna')

In [215]:
addDatetime = data_preperation.GenerateDatetime()
filled_na_df.index = addDatetime.data_cleaner(data=filled_na_df, start_date=datetime(2022,1,1), freq='D')

In [216]:
filled_na_df

,A,B,C
2022-01-01,10.000000,50.000000,100.000000
2022-01-02,15.000000,55.000000,150.000000
2022-01-03,12.000000,52.000000,120.000000
2022-01-04,12.222222,54.000000,140.000000
2022-01-05,20.000000,60.000000,200.000000
2022-01-06,8.000000,48.000000,80.000000
2022-01-07,11.000000,52.555556,110.000000
2022-01-08,13.000000,53.000000,130.000000
2022-01-09,16.000000,56.000000,160.000000
2022-01-10,5.000000,45.000000,132.222222
